# 第5章：使用Numba编写CUDA内核

<img src="images/chapter-05/numba_title.png" style="width:442px;"/>

Numba是一个开源的JIT编译器,可以将Python和NumPy代码的子集转换为快速的机器代码。

Numba通过直接将受限的Python代码子集编译成CUDA内核和设备函数来支持CUDA GPU编程,遵循CUDA执行模型。用Numba编写的内核可以直接访问NumPy数组。NumPy数组在CPU和GPU之间自动传输。


## Numba基础

Numba通过直接将受限的Python代码子集编译成CUDA内核和设备函数来支持CUDA GPU编程,遵循CUDA执行模型。用Numba编写的内核可以直接访问NumPy数组。NumPy数组在CPU和GPU之间自动传输。Numba的集成编译系统允许使用CPU和GPU的特性创建代码,而不需要对Python语言进行太多更改。

### 安装

在设置Numba编程环境之前,首先确保您满足以下先决条件(如果您按照安装CuPy的说明操作,可以跳过这些步骤)：
- CUDA兼容的GPU。(参见https://developer.nvidia.com/cuda-gpus 获取NVIDIA GPU列表)
- CUDA兼容的NVIDIA驱动程序。
- CUDA工具包

查看安装说明：https://numba.pydata.org/numba-doc/latest/user/installing.html 

### 使用`@cuda.jit`创建内核函数

在Numba中,`@jit`装饰器用于指定要由Numba即时编译器优化的函数。在GPU的上下文中,我们使用名为`@cuda.jit`的版本来指定要由GPU上的多个线程同时执行的内核函数进行优化。

```python
from numba import cuda

@cuda.jit
def foo(input_array, output_array):
    # 代码块在这里

这应该看起来与在CPU上使用numba非常相似。


### 启动内核函数

在运行内核函数之前,需要指定块数和每个块的线程数。这将定义执行网格的形状。

```python
@cuda.jit
def foo(input_array, output_array):
    # 1D块中的线程ID
    thread_id = cuda.threadIdx.x
    # 1D网格中的块ID
    block_id = cuda.blockIdx.x
    # 块宽度,即每个块的线程数
    block_width = cuda.blockDim.x
    # 计算数组内的扁平化索引
    i = thread_id + block_id * block_width
    if i < an_array.size:  # 检查数组边界
        output_array[i] = input_array[i]
```

要调用`foo()`函数,我们必须指定块和网格大小。

```python
input = np.asarray(range(10))
output = np.zeros(len(input))


block_threads = 32
grid_blocks = (input.size + (block_threads - 1)) // block_threads

foo[grid_blocks, block_threads](input, output)
```

对于简单的示例,`cuda.grid()`函数是管理线程、块和网格的便捷方式。完整的脚本可以这样重写：

```python
import numpy as np
from numba import cuda

input = np.asarray(range(10))
output = np.zeros(len(input))

@cuda.jit
def foo(input_array, output_array):
    i = cuda.grid(1)
    output_array[i] = input_array[i]
    
foo[1, len(input)](input, output)

output
```

注意：当CUDA内核执行时,调用会在GPU完成之前立即返回。然后需要同步内核执行,以确保结果被传回CPU。如果不完成此步骤,您可能会遇到内存错误,因为后续调用试图读取或写入受限内存。使用cuda.synchronize()确保数据一致性。


### 指定线程数和块数

现在不要太担心这个。只需记住我们需要指定要调用内核的次数,这是通过两个相乘得到总网格大小的数字给出的。这种设置将确保网格大小有足够的线程来处理数据大小,即使该数字不是每个块的线程数的精确倍数。

每个块的线程数经验法则：
- 最佳块大小通常是32(warp大小)的倍数。
- 需要分析和基准测试来确定最佳值。

入门：
- NSight的占用率计算器：https://docs.nvidia.com/nsight-compute/NsightCompute/index.html#occupancy-calculator)
- 多个来源建议从128到256之间的数字开始调优。

块和网格维度会影响CUDA性能。较大的块可以导致更好地利用共享内存并减少启动许多小块的开销。但是,过大的块可能会减少可以并发执行的块数量,这将导致GPU利用率不足。找到正确的平衡对于利用GPU是必要的。


## Numba与CuPy

CuPy的`cupy.ndarray`实现了`__cuda_array_interface__`,这是与Numba v0.39.0或更高版本兼容的CUDA数组交换接口(详见Numba的CUDA数组接口 https://numba.readthedocs.io/en/stable/cuda/cuda_array_interface.html)。这意味着您可以将CuPy数组传递给用Numba JIT编译的内核。

在此示例中,我们使用`cupy`数组而不是`numpy`数组：

```python
import cupy
from numba import cuda

@cuda.jit
def add(x_array, y_array, output_array):
        start = cuda.grid(1)
        stride = cuda.gridsize(1)
        for i in range(start, x.shape[0], stride):
                output_array[i] = x_array[i] + y_array[i]

a = cupy.arange(10)
b = a * 2
out = cupy.zeros_like(a)

add[1, 32](a, b, out)

print(out)  # => [ 0  3  6  9 12 15 18 21 24 27]
```


## 有用的参考链接
Numba用于CUDA GPU：https://numba.pydata.org/numba-doc/latest/cuda/index.html 

CuPy的互操作性指南(包括Numba)：https://docs.cupy.dev/en/stable/user_guide/interoperability.html 

Numba Github仓库：https://github.com/numba/numba 


# 示例：

## 定义和启动内核函数

In [ ]:
import numpy as np
from numba import cuda

input = np.asarray(range(10))
output = np.zeros(len(input))

@cuda.jit
def foo(input_array, output_array):
    # 1D块中的线程ID
    thread_id = cuda.threadIdx.x
    # 1D网格中的块ID
    block_id = cuda.blockIdx.x
    # 块宽度,即每个块的线程数
    block_width = cuda.blockDim.x
    # 计算数组内的扁平化索引
    i = thread_id + block_id * block_width
    if i < an_array.size:  # 检查数组边界
        output_array[i] = input_array[i]

block_threads = 32
grid_blocks = (input.size + (block_threads - 1)) // block_threads

foo[grid_blocks, block_threads](input, output)

output

## 使用grid()简化内核函数

In [ ]:
import numpy as np
from numba import cuda

input = np.asarray(range(10))
output = np.zeros(len(input))

@cuda.jit
def foo(input_array, output_array):
    i = cuda.grid(1)
    output_array[i] = input_array[i]
    
foo[1, len(input)](input, output)

output

## 将Numba与CuPy一起使用

In [ ]:
import cupy
from numba import cuda

@cuda.jit
def add(x_array, y_array, output_array):
        start = cuda.grid(1)
        stride = cuda.gridsize(1)
        for i in range(start, x.shape[0], stride):
                output_array[i] = x_array[i] + y_array[i]

a = cupy.arange(10)
b = a * 2
out = cupy.zeros_like(a)

add[1, 32](a, b, out)

print(out)  # => [ 0  3  6  9 12 15 18 21 24 27]